<a href="https://colab.research.google.com/github/mario-i-caicedo-ai/University_Physics/blob/main/jupyter_notebooks/Left_and_Right_Movers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Left and Right Movers

A simple jupyter notebook coded by Gemini  under the propmting of Mario I. Caicedo

The purpose of this notebook is twofold,

* Introduce the ideas of left and right movers to sophomore students whio are taking Physics For Sciences and Engineering Course and

* Show said students the magic of Python, jupyter notebooks and colabe.




## Cell 1: Imports

These are the libraries needed for numerical operations, plotting, animation, and interactive widgets.

In [ ]:
# Import the necessary libraries
import numpy as np # For numerical operations (like linspace, exp)
import matplotlib.pyplot as plt # For creating plots
import matplotlib.animation as animation # For creating animations
from IPython.display import HTML, display, clear_output # For displaying animations and managing output in Colab
import ipywidgets as widgets # For creating interactive elements like sliders
from ipywidgets import interact, FloatSlider, VBox, Layout # Specific ipywidgets components

----------------------
## Cell 2: Define the Pulse Shape Function

This function defines the shape of our disturbance. We choose a -non normalized- Gaussian function
$$
G(x,t,v)=e^{\frac{(x-vt)^2}{2\sigma^2}}
$$

The notation is selfexplanatory


In [ ]:
# Define the mathematical function that represents our pulse.
# We'll use a Gaussian pulse, which is a common and smooth wave shape.
# The (x - v*t) term is what makes the pulse move along the x-axis.
def gaussian_pulse(x, t, v, A, sigma):
    """
    Generates a Gaussian pulse at a given position and time.

    Args:
        x (numpy.ndarray): Array of spatial points (e.g., the x-axis).
        t (float): The current moment in time.
        v (float): The velocity of the pulse (physical velocity).
        A (float): The maximum amplitude (height) of the pulse.
        sigma (float): Controls the width (spread) of the pulse.

    Returns:
        numpy.ndarray: The amplitude u(x,t) of the pulse at each x-point for the given time.
    """
    return A * np.exp(-((x - v * t)**2) / (2 * sigma**2))

---------------------------------
## Cell 3: Define Animation Parameters

These constants control the visual aspects and speed of the animation.

In [ ]:
# --- Parameters that define our pulse and the animation's scope ---

# Pulse characteristics
A = 1.0        # Amplitude of the pulse (its maximum height)
sigma = 0.5    # Width of the pulse (smaller sigma = narrower pulse)

# Spatial domain characteristics
x_min = -10.0  # Leftmost x-value displayed on the plot
x_max = 10.0   # Rightmost x-value displayed on the plot
num_x_points = 500 # Number of points to calculate the pulse shape across the x-axis.
                     # More points make the line smoother.

# Time domain characteristics
t_min = 0.0    # Start time for the animation
t_max = 10.0   # End time for the animation
num_frames = 200 # Total number of individual pictures (frames) in the animation.
                     # More frames make the animation appear smoother.
fps = 30       # Frames per second. Higher FPS makes the animation faster.

# Create the array of x-values where we will evaluate the pulse
x = np.linspace(x_min, x_max, num_x_points)

# Create the array of time values, one for each frame
t_frames = np.linspace(t_min, t_max, num_frames)

# This global variable will store the current velocity value from our slider.
# It's 'global' so our animation function can access it directly.
global_v_value = 1.0 # Initial velocity for the pulse when the animation first loads

----------------------------
## Cell 4: Initial Plot Setup

Explain that this sets up the empty graph where the pulse will be drawn and updated. line, = ax.plot() creates a "placeholder" for the changing data.

In [ ]:
# --- Set up the initial state of the plot ---

# Create a figure and a set of subplots (here, just one plot)
fig, ax = plt.subplots(figsize=(10, 6))

# Plot the initial state of the pulse (at t_frames[0] and global_v_value).
# 'line,' is important: it unpacks the single line object from the list returned by plot().
line, = ax.plot(x, gaussian_pulse(x, t_frames[0], global_v_value, A, sigma), 'r-', label='Pulse')

# Set the fixed limits for the x and y axes
ax.set_xlim(x_min, x_max)
ax.set_ylim(-0.2, A * 1.1) # Y-limit slightly above amplitude for better visualization

# Add labels and a title (the title will be updated dynamically later)
ax.set_xlabel('Position (x)')
ax.set_ylabel('Amplitude u(x,t)')
ax.set_title(f'Propagating Gaussian Pulse (v = {global_v_value:.1f} m/s)') # Initial title
ax.grid(True) # Add a grid for better readability
ax.legend() # Display the label defined in ax.plot()

# Add a text element to display the current time on the plot itself (optional)
time_text = ax.text(0.05, 0.95, '', transform=ax.transAxes, va='top', ha='left', fontsize=12)

# It's crucial to close the matplotlib figure here.
# This prevents it from displaying as a static plot *outside* the interactive output.
plt.close(fig)

----------------------
## Cell 5: Animation Update Function

This function is called repeatedly by the animation engine. Its job is to calculate the new pulse position for the current time and update the plot's data.


In [ ]:
# --- The 'animate' function: This is the core of the animation ---
# FuncAnimation will call this function for each frame.
# 'i' is the current frame number (from 0 to num_frames - 1).
def animate(i):
    # Get the current time for this frame
    current_time = t_frames[i]

    # Calculate the pulse's y-values at this time, using the global velocity.
    y_data = gaussian_pulse(x, current_time, global_v_value, A, sigma)

    # Update the y-data of the 'line' object on the plot
    line.set_ydata(y_data)

    # Update the time display text on the plot
    time_text.set_text(f'Time: {current_time:.2f} s')

    # Update the title to reflect the current velocity (since global_v_value can change)
    ax.set_title(f'Propagating Gaussian Pulse (v = {global_v_value:.1f} m/s)')

    # Return all the plot elements (artists) that were modified.
    # This is important for 'blit=True' optimization.
    return line, time_text, ax.title

-------------------------------------------
## Cell 6: Interactive Slider Setup

This cell creates the slider widget and defines what happens when the slider is moved. The on_v_slider_change function updates our global_v_value.

In [ ]:
# --- Create the ipywidgets Slider for interactive control of velocity ---

# Define the slider widget itself.
v_slider = FloatSlider(
    min=-2.0,       # Minimum velocity value the slider can set
    max=2.0,        # Maximum velocity value the slider can set
    step=0.1,       # The increment/decrement step when moving the slider
    value=global_v_value, # The initial position/value of the slider
    description='Phase Velocity (v):', # Label displayed next to the slider
    orientation='horizontal', # Slider orientation
    readout=True,   # Display the current numerical value next to the slider
    readout_format='.1f', # Format for the displayed value (1 decimal place)
    layout=Layout(width='auto') # Allows the slider width to adjust automatically
)

# --- Define the Callback Function for the Slider ---
# This function is automatically called by ipywidgets whenever the slider's value changes.
def on_v_slider_change(change):
    # Declare that we intend to modify the global variable 'global_v_value'.
    global global_v_value

    # Update the global velocity variable with the new value from the slider.
    global_v_value = change.new
    # IMPORTANT: We do NOT re-run the animation or the plot here.
    # The 'animate' function (which is continuously running) will automatically
    # pick up this new 'global_v_value' in its next frame update.

# Link the slider to the callback function.
# 'observe' means: when the 'value' property of 'v_slider' changes, call 'on_v_slider_change'.
v_slider.observe(on_v_slider_change, names='value')

------------------------------
## Cell 7: Create the Animation Object

This line puts everything together: the figure, the update function, the frames, and timing. The animation itself starts "running" internally here.)

In [ ]:
# --- Create the animation object ---
# This line orchestrates the animation process.
# - fig: The matplotlib figure object to animate.
# - animate: The function that updates each frame.
# - frames: The number of frames in the animation.
# - interval: The delay between frames in milliseconds (1000/fps for smooth playback).
# - blit=True: Optimizes drawing by only updating parts of the plot that have changed.
#              (Can sometimes be tricky with global variables; if issues, try blit=False).
ani = animation.FuncAnimation(fig, animate, frames=num_frames, interval=1000/fps, blit=True)

----------------------------------

## Cell 8: Display the Animation in the Notebook

(Explain that this is the final step to make the slider and the animation visible in your Colab output. You run this cell after defining everything above.)



In [ ]:
# --- Display the interactive widgets and the animation ---

# Create an output widget. This acts as a container for our matplotlib animation.
animation_output = widgets.Output()

# Arrange the slider and the animation output vertically.
# 'ui' is the user interface structure that will be displayed.
ui = VBox([v_slider, animation_output])

# Display the entire UI structure (slider + animation container).
display(ui)

# Now, crucially, we tell the animation to render itself *into* our 'animation_output' container.
# This makes the animation appear below the slider.
with animation_output:
    display(HTML(ani.to_jshtml()))

# Optional: If you also want to save the animation to an MP4 file
# (remember this is a separate, non-interactive save process):
# video_path = 'moving_pulse.mp4'
# print(f"Saving animation to {video_path} (this might take a moment)...")
# try:
#     ani.save(video_path, writer='ffmpeg', fps=fps)
#     print(f"Animation saved successfully to {video_path}")
# except Exception as e:
#     print(f"Error saving animation: {e}")
#     print("Ensure ffmpeg is available (it should be in Colab).")

# Houston Community College

Replace the following link with the actual URL of the Houston Community College logo.

![Houston Community College Logo](URL_OF_THE_LOGO_IMAGE)